In [24]:
import pandas as pd
import numpy as np
print('Success')

Success


# Importing data

In [25]:
# Importing data

melbourne_file_path = './melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)

# Basic data exploration

In [26]:
# melbourne_data.shape

In [58]:
# melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0
6,Abbotsford,124 Yarra St,3,h,1876000.0,S,Nelson,7/05/2016,2.5,3067.0,...,2.0,0.0,245.0,210.0,1910.0,Yarra,-37.8024,144.9993,Northern Metropolitan,4019.0
7,Abbotsford,98 Charles St,2,h,1636000.0,S,Nelson,8/10/2016,2.5,3067.0,...,1.0,2.0,256.0,107.0,1890.0,Yarra,-37.8060,144.9954,Northern Metropolitan,4019.0


In [28]:
# melbourne_data.tail()

In [29]:
# melbourne_data.describe()

In [30]:
melbourne_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

# Preparing data

In [31]:
melbourne_data = melbourne_data.dropna(axis=0)

In [32]:
# Setting prediction target

y = melbourne_data.Price

In [33]:
# Choosing features

# melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = melbourne_data[melbourne_features]

# X.describe()
# X.head()

# Building a Decision Tree Regressor model

Steps to building and using a model:
1. Define the model
2. Fit the data
3. Predict target values
4. Evaluate the predictions

In [34]:
# Using the Decision Tree Regressor model

from sklearn.tree import DecisionTreeRegressor

# Define model. Specify a number for random_state to ensure same results each run
melbourne_model = DecisionTreeRegressor(random_state=1)

In [35]:
# Fit model
melbourne_model.fit(X, y)

DecisionTreeRegressor(random_state=1)

In [36]:
# Predicting the target values

# Make predictions for the first 5 houses
melbourne_model.predict(X.head())

array([1035000., 1465000., 1600000., 1876000., 1636000.])

# Validating the model

In [42]:
# Using in-sample data (incorrect method of training model)

from sklearn.metrics import mean_absolute_error

# Predicting price for entire dataset
predicted_home_prices = melbourne_model.predict(X)

# Validating the model (using the training set)
mean_absolute_error(y, predicted_home_prices)

62426.08828276307

In [43]:
# Using out-of-sample data (correct method of training model)

from sklearn.model_selection import train_test_split

# Split the model into training and validation subsets
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=2)

# Fit the model to the training data
melbourne_model.fit(train_X, train_y)

# Predict from test data using model we just trained
val_predictions = melbourne_model.predict(val_X)

# Get error of the model
val_error = mean_absolute_error(val_y, val_predictions)
print(val_error)

248894.15300193673


# Underfitting and overfitting

In [44]:
# Function to train and validate model

def get_mae(max_leaf_nodes, train_X, train_y, val_X, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    val_prediction = model.predict(val_X)
    mae = mean_absolute_error(val_y, val_prediction)
    return mae

In [52]:
# Cycle through various amounts of nodes in tree, output best number

leaf_nodes_array = [5, 50, 500, 5000]
best_leaf_nodes, best_mae = 0, 10000000
for max_leaf_nodes in leaf_nodes_array:
    mae = get_mae(max_leaf_nodes, train_X, train_y, val_X, val_y)
    print(f'Max leaf nodes: {max_leaf_nodes}; MAE: {mae.round(2)}')
    if mae < best_mae:
        best_leaf_nodes = max_leaf_nodes
        best_mae = mae
    
print(best_leaf_nodes)

Max leaf nodes: 5; MAE: 321694.85
Max leaf nodes: 50; MAE: 264918.08
Max leaf nodes: 500; MAE: 239626.99
Max leaf nodes: 5000; MAE: 252354.33
500


# Random Forests model

In [57]:
from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
forest_model_predictions = forest_model.predict(val_X)
forest_model_mae = mean_absolute_error(val_y, forest_model_predictions)
print(forest_model_mae)

175311.75567186202


# Generate a submission for Kaggle

In [ ]:
output = pd.DataFrame({'ID': melb_data.ID,
                       'SalePrice': test_preds})

output.to_csv('submission.csv', index=False)